In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision
from pathlib import Path
import time
import copy
import shutil
import os

In [3]:
torch.__version__

'2.9.0+cu128'

In [4]:
#data_paths
 
input_path = Path("data/datasets/trashnet_01/")
input_path2 = Path("data/datasets/self-collected/")

In [5]:
#function to save models later on

model_dir = "trained_models"
os.makedirs(model_dir, exist_ok=True)

def save_model(model, model_name, history=None):
    save_path = os.path.join(model_dir, f"{model_name}.pth")

    checkpoint = {
        "model_state_dict": model.state_dict(),
    }

    if history is not None:
        checkpoint["history"] = history

    torch.save(checkpoint, save_path)
    print(f"Model saved")    

In [6]:
import random

seed = 16

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
#calculate class weighted cross entropy loss

from collections import Counter

def compute_class_weights_from_imagefolder(train_root, num_classes):
    ds = datasets.ImageFolder(train_root, transform=transforms.ToTensor())
    counts = Counter(ds.targets)
    total = sum(counts.values())

    weights = [total / counts[i] for i in range(num_classes)]
    weights = torch.tensor(weights, dtype=torch.float)
    weights = weights / weights.sum() * num_classes 
    return weights, ds.classes, counts

tmp_ds = datasets.ImageFolder(input_path / "train")
num_classes = len(tmp_ds.classes)

class_weights, class_names, train_counts = compute_class_weights_from_imagefolder(
    input_path / "train",
    num_classes
)

class_weights = class_weights.to(device)

criterion_weighted = torch.nn.CrossEntropyLoss(weight=class_weights)

print("Classes:", class_names)
print("Train counts:", train_counts)
print("Class weights:", class_weights)

Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Train counts: Counter({3: 475, 1: 400, 4: 385, 2: 328, 0: 322, 5: 109})
Class weights: tensor([0.8270, 0.6657, 0.8119, 0.5606, 0.6917, 2.4431], device='cuda:0')


In [9]:
# baseline data transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

data_transforms = {

    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),  
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(),
        normalize
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),    
}

image_datasets = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms["val"]),

    "test_01": datasets.ImageFolder(input_path2, data_transforms["val"])
}


dataloaders = {
    "train": torch.utils.data.DataLoader(image_datasets["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets["test"], batch_size=32, shuffle=False, num_workers=4),

    "test_01": torch.utils.data.DataLoader(image_datasets["test_01"], batch_size=32, shuffle=False, num_workers=4)
}   


dataset_size = {
    "train": len(image_datasets["train"]),
    "val": len(image_datasets["val"]),
    "test_01": len(image_datasets["test_01"])
}

num_classes = len((image_datasets["train"]).classes)

In [10]:
#geometric augmentation

data_transforms_geo = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1), shear=5),
        transforms.ToTensor(),
        normalize
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]), 
}

image_datasets_geo = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms_geo["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms_geo["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms_geo["val"]),
    "test_01": datasets.ImageFolder(input_path2, data_transforms_geo["val"])
}

dataloaders_geo = {
    "train": torch.utils.data.DataLoader(image_datasets_geo["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets_geo["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets_geo["test"], batch_size=32, shuffle=False, num_workers=4),
    "test_01": torch.utils.data.DataLoader(image_datasets_geo["test_01"], batch_size=32, shuffle=False, num_workers=4)
}

dataset_size_geo = {
    "train": len(image_datasets_geo["train"]),
    "val": len(image_datasets_geo["val"]),
    "test": len(image_datasets_geo["test"]),
    "test_01": len(image_datasets_geo["test_01"])
}

num_classes_geo = len(image_datasets_geo["train"].classes)


In [11]:
#photometric
 
data_transforms_photo = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        normalize
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]), 
}

image_datasets_photo = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms_photo["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms_photo["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms_photo["val"]),
    "test_01": datasets.ImageFolder(input_path2, data_transforms_photo["val"])
}

dataloaders_photo = {
    "train": torch.utils.data.DataLoader(image_datasets_photo["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets_photo["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets_photo["test"], batch_size=32, shuffle=False, num_workers=4),
    "test_01": torch.utils.data.DataLoader(image_datasets_photo["test_01"], batch_size=32, shuffle=False, num_workers=4)
}

dataset_size_photo = {
    "train": len(image_datasets_photo["train"]),
    "val": len(image_datasets_photo["val"]),
    "test": len(image_datasets_photo["test"]),
    "test_01": len(image_datasets_photo["test_01"])
}

num_classes_photo = len(image_datasets_photo["train"].classes)


In [12]:
#aug mix
 
data_transforms_mix = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1), shear=5),
        transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        normalize,
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value='random')
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]), 
}

image_datasets_mix = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms_mix["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms_mix["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms_mix["val"]),
    "test_01": datasets.ImageFolder(input_path2, data_transforms_mix["val"])
}

dataloaders_mix = {
    "train": torch.utils.data.DataLoader(image_datasets_mix["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets_mix["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets_mix["test"], batch_size=32, shuffle=False, num_workers=4),
    "test_01": torch.utils.data.DataLoader(image_datasets_mix["test_01"], batch_size=32, shuffle=False, num_workers=4)
}

dataset_size_mix = {
    "train": len(image_datasets_mix["train"]),
    "val": len(image_datasets_mix["val"]),
    "test": len(image_datasets_mix["test"]),
    "test_01": len(image_datasets_mix["test_01"])
}

num_classes_mix = len(image_datasets_mix["train"].classes)


In [13]:
def train_model(model, dataloaders, dataset_size, device,
                criterion, optimizer, num_epochs=25):

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": [],
    }

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels).item()

            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects / dataset_size[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == "train":
                history["train_loss"].append(epoch_loss)
                history["train_acc"].append(epoch_acc)
            else:
                history["val_loss"].append(epoch_loss)
                history["val_acc"].append(epoch_acc)

            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val Acc: {best_acc:.4f}")

    model.load_state_dict(best_model_wts)
    return model, history


In [65]:
#RESNET-50

resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes) #Replace last classifier layer
resnet = resnet.to(device)

optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

resnet, resnet_history = train_model(
    model=resnet,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)


Epoch 1/25
----------
train Loss: 1.6589 Acc: 0.4373
val Loss: 1.4070 Acc: 0.7371

Epoch 2/25
----------
train Loss: 1.2310 Acc: 0.6637
val Loss: 0.8990 Acc: 0.7928

Epoch 3/25
----------
train Loss: 0.9011 Acc: 0.7271
val Loss: 0.5952 Acc: 0.8127

Epoch 4/25
----------
train Loss: 0.7158 Acc: 0.7667
val Loss: 0.4550 Acc: 0.8645

Epoch 5/25
----------
train Loss: 0.6185 Acc: 0.7935
val Loss: 0.3681 Acc: 0.8765

Epoch 6/25
----------
train Loss: 0.5148 Acc: 0.8291
val Loss: 0.3096 Acc: 0.9124

Epoch 7/25
----------
train Loss: 0.4570 Acc: 0.8529
val Loss: 0.2731 Acc: 0.9124

Epoch 8/25
----------
train Loss: 0.4087 Acc: 0.8628
val Loss: 0.2841 Acc: 0.9243

Epoch 9/25
----------
train Loss: 0.3750 Acc: 0.8747
val Loss: 0.2173 Acc: 0.9522

Epoch 10/25
----------
train Loss: 0.3494 Acc: 0.8886
val Loss: 0.2131 Acc: 0.9243

Epoch 11/25
----------
train Loss: 0.3288 Acc: 0.8940
val Loss: 0.2074 Acc: 0.9363

Epoch 12/25
----------
train Loss: 0.2756 Acc: 0.9188
val Loss: 0.1810 Acc: 0.9363

E

In [66]:
save_model(resnet, "resnet50_baseline", resnet_history)

del resnet
del optimizer
del resnet_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [67]:
#RESNET-50 photo augmentations

resnet_aug_1 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet_aug_1.fc = nn.Linear(resnet_aug_1.fc.in_features, num_classes) #Replace last classifier layer
resnet_aug_1 = resnet_aug_1.to(device)

optimizer = optim.SGD(resnet_aug_1.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

resnet_aug_1, resnet_history_aug_1 = train_model(
    model=resnet_aug_1,
    dataloaders=dataloaders_photo,
    dataset_size=dataset_size_photo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)


Epoch 1/25
----------
train Loss: 1.6466 Acc: 0.4284
val Loss: 1.3709 Acc: 0.7092

Epoch 2/25
----------
train Loss: 1.2710 Acc: 0.6478
val Loss: 0.9680 Acc: 0.7689

Epoch 3/25
----------
train Loss: 0.9658 Acc: 0.7043
val Loss: 0.6172 Acc: 0.8486

Epoch 4/25
----------
train Loss: 0.8018 Acc: 0.7370
val Loss: 0.4815 Acc: 0.8685

Epoch 5/25
----------
train Loss: 0.6979 Acc: 0.7717
val Loss: 0.4554 Acc: 0.8924

Epoch 6/25
----------
train Loss: 0.6394 Acc: 0.7885
val Loss: 0.3583 Acc: 0.9163

Epoch 7/25
----------
train Loss: 0.5432 Acc: 0.8227
val Loss: 0.3236 Acc: 0.9044

Epoch 8/25
----------
train Loss: 0.5207 Acc: 0.8167
val Loss: 0.2902 Acc: 0.9124

Epoch 9/25
----------
train Loss: 0.4796 Acc: 0.8336
val Loss: 0.2684 Acc: 0.9084

Epoch 10/25
----------
train Loss: 0.4419 Acc: 0.8430
val Loss: 0.2782 Acc: 0.9243

Epoch 11/25
----------
train Loss: 0.3906 Acc: 0.8712
val Loss: 0.2299 Acc: 0.9442

Epoch 12/25
----------
train Loss: 0.3678 Acc: 0.8747
val Loss: 0.2351 Acc: 0.9363

E

In [68]:
save_model(resnet_aug_1, "resnet50_photo", resnet_history_aug_1)

del resnet_aug_1
del optimizer
del resnet_history_aug_1
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [69]:
#RESNET-50 geo augmentations

resnet_aug_2 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet_aug_2.fc = nn.Linear(resnet_aug_2.fc.in_features, num_classes) #Replace last classifier layer
resnet_aug_2 = resnet_aug_2.to(device)

optimizer = optim.SGD(resnet_aug_2.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

resnet_aug_2, resnet_history_aug_2 = train_model(
    model=resnet_aug_2,
    dataloaders=dataloaders_geo,
    dataset_size=dataset_size_geo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)


Epoch 1/25
----------
train Loss: 1.6671 Acc: 0.4329
val Loss: 1.4159 Acc: 0.6892

Epoch 2/25
----------
train Loss: 1.3044 Acc: 0.6285
val Loss: 1.0671 Acc: 0.7450

Epoch 3/25
----------
train Loss: 0.9916 Acc: 0.6954
val Loss: 0.7522 Acc: 0.8048

Epoch 4/25
----------
train Loss: 0.7981 Acc: 0.7415
val Loss: 0.5655 Acc: 0.8446

Epoch 5/25
----------
train Loss: 0.6795 Acc: 0.7746
val Loss: 0.4839 Acc: 0.8645

Epoch 6/25
----------
train Loss: 0.5741 Acc: 0.8019
val Loss: 0.3987 Acc: 0.8845

Epoch 7/25
----------
train Loss: 0.5051 Acc: 0.8326
val Loss: 0.3530 Acc: 0.8805

Epoch 8/25
----------
train Loss: 0.4560 Acc: 0.8460
val Loss: 0.3163 Acc: 0.8924

Epoch 9/25
----------
train Loss: 0.4132 Acc: 0.8569
val Loss: 0.2937 Acc: 0.9084

Epoch 10/25
----------
train Loss: 0.4142 Acc: 0.8648
val Loss: 0.3124 Acc: 0.8884

Epoch 11/25
----------
train Loss: 0.3696 Acc: 0.8687
val Loss: 0.2311 Acc: 0.9044

Epoch 12/25
----------
train Loss: 0.3611 Acc: 0.8762
val Loss: 0.2296 Acc: 0.9203

E

In [70]:
save_model(resnet_aug_2, "resnet50_geo", resnet_history_aug_2)

del resnet_aug_2
del optimizer
del resnet_history_aug_2
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [71]:
#RESNET-50 mixed augmentations

resnet_mixed = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet_mixed.fc = nn.Linear(resnet_mixed.fc.in_features, num_classes) #Replace last classifier layer
resnet_mixed = resnet_mixed.to(device)

optimizer = optim.SGD(resnet_mixed.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

resnet_mixed, resnet_mixed_history = train_model(
    model=resnet_mixed,
    dataloaders=dataloaders_mix,
    dataset_size=dataset_size_mix,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)


Epoch 1/25
----------
train Loss: 1.7256 Acc: 0.3304
val Loss: 1.5572 Acc: 0.6096

Epoch 2/25
----------
train Loss: 1.4937 Acc: 0.5518
val Loss: 1.2048 Acc: 0.7371

Epoch 3/25
----------
train Loss: 1.2157 Acc: 0.6424
val Loss: 0.8552 Acc: 0.7888

Epoch 4/25
----------
train Loss: 0.9930 Acc: 0.6830
val Loss: 0.6633 Acc: 0.8167

Epoch 5/25
----------
train Loss: 0.8643 Acc: 0.7207
val Loss: 0.5502 Acc: 0.8287

Epoch 6/25
----------
train Loss: 0.7564 Acc: 0.7533
val Loss: 0.5031 Acc: 0.8566

Epoch 7/25
----------
train Loss: 0.7049 Acc: 0.7578
val Loss: 0.4041 Acc: 0.8645

Epoch 8/25
----------
train Loss: 0.6377 Acc: 0.7796
val Loss: 0.3989 Acc: 0.8765

Epoch 9/25
----------
train Loss: 0.6152 Acc: 0.7850
val Loss: 0.3771 Acc: 0.8725

Epoch 10/25
----------
train Loss: 0.5544 Acc: 0.8009
val Loss: 0.3198 Acc: 0.8805

Epoch 11/25
----------
train Loss: 0.5268 Acc: 0.8128
val Loss: 0.3008 Acc: 0.8924

Epoch 12/25
----------
train Loss: 0.4982 Acc: 0.8266
val Loss: 0.2746 Acc: 0.9084

E

In [72]:
save_model(resnet_mixed, "resnet50_mixed", resnet_mixed_history)

del resnet_mixed
del optimizer
del resnet_mixed_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [73]:
#densenet

densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT) #Load pretrained model
densenet.classifier = nn.Linear(densenet.classifier.in_features, num_classes)
densenet = densenet.to(device)

optimizer = optim.SGD(densenet.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

densenet, densenet_history = train_model(
    model=densenet,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.3142 Acc: 0.5102
val Loss: 0.7171 Acc: 0.7530

Epoch 2/25
----------
train Loss: 0.6958 Acc: 0.7722
val Loss: 0.4193 Acc: 0.8446

Epoch 3/25
----------
train Loss: 0.5284 Acc: 0.8202
val Loss: 0.3151 Acc: 0.8924

Epoch 4/25
----------
train Loss: 0.4160 Acc: 0.8539
val Loss: 0.2690 Acc: 0.9163

Epoch 5/25
----------
train Loss: 0.3592 Acc: 0.8796
val Loss: 0.2175 Acc: 0.9402

Epoch 6/25
----------
train Loss: 0.3458 Acc: 0.8772
val Loss: 0.2734 Acc: 0.8964

Epoch 7/25
----------
train Loss: 0.3074 Acc: 0.8990
val Loss: 0.2262 Acc: 0.9163

Epoch 8/25
----------
train Loss: 0.2531 Acc: 0.9203
val Loss: 0.2111 Acc: 0.9482

Epoch 9/25
----------
train Loss: 0.2347 Acc: 0.9128
val Loss: 0.2940 Acc: 0.9084

Epoch 10/25
----------
train Loss: 0.2124 Acc: 0.9227
val Loss: 0.1986 Acc: 0.9203

Epoch 11/25
----------
train Loss: 0.2229 Acc: 0.9222
val Loss: 0.1839 Acc: 0.9163

Epoch 12/25
----------
train Loss: 0.1890 Acc: 0.9331
val Loss: 0.2115 Acc: 0.9283

E

In [74]:
save_model(densenet, "densenet121_baseline", densenet_history)

del densenet
del optimizer
del densenet_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [75]:
#densenet geometric augmentation

densenet_geo = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT) #Load pretrained model
densenet_geo.classifier = nn.Linear(densenet_geo.classifier.in_features, num_classes)
densenet_geo = densenet_geo.to(device)

optimizer = optim.SGD(densenet_geo.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

densenet_geo, densenet_geo_history = train_model(
    model=densenet_geo,
    dataloaders=dataloaders_geo,
    dataset_size=dataset_size_geo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.3991 Acc: 0.4572
val Loss: 0.8023 Acc: 0.7729

Epoch 2/25
----------
train Loss: 0.8437 Acc: 0.7038
val Loss: 0.4657 Acc: 0.8645

Epoch 3/25
----------
train Loss: 0.6190 Acc: 0.7806
val Loss: 0.4394 Acc: 0.8446

Epoch 4/25
----------
train Loss: 0.5211 Acc: 0.8187
val Loss: 0.3183 Acc: 0.9004

Epoch 5/25
----------
train Loss: 0.4307 Acc: 0.8425
val Loss: 0.2861 Acc: 0.9163

Epoch 6/25
----------
train Loss: 0.3880 Acc: 0.8653
val Loss: 0.2590 Acc: 0.9124

Epoch 7/25
----------
train Loss: 0.3640 Acc: 0.8782
val Loss: 0.2698 Acc: 0.9004

Epoch 8/25
----------
train Loss: 0.3578 Acc: 0.8668
val Loss: 0.2076 Acc: 0.9243

Epoch 9/25
----------
train Loss: 0.3239 Acc: 0.8886
val Loss: 0.1920 Acc: 0.9363

Epoch 10/25
----------
train Loss: 0.3019 Acc: 0.8930
val Loss: 0.2088 Acc: 0.9522

Epoch 11/25
----------
train Loss: 0.3026 Acc: 0.8900
val Loss: 0.2268 Acc: 0.9124

Epoch 12/25
----------
train Loss: 0.2762 Acc: 0.9004
val Loss: 0.2229 Acc: 0.9323

E

In [76]:
save_model(densenet_geo, "densenet121_geo", densenet_geo_history)

del densenet_geo
del optimizer
del densenet_geo_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [77]:
#densenet photometric augmentation

densenet_photo = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT) #Load pretrained model
densenet_photo.classifier = nn.Linear(densenet_photo.classifier.in_features, num_classes)
densenet_photo = densenet_photo.to(device)

optimizer = optim.SGD(densenet_photo.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

densenet_photo, densenet_photo_history = train_model(
    model=densenet_photo,
    dataloaders=dataloaders_photo,
    dataset_size=dataset_size_photo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.3817 Acc: 0.4770
val Loss: 0.7113 Acc: 0.7769

Epoch 2/25
----------
train Loss: 0.8041 Acc: 0.7370
val Loss: 0.4674 Acc: 0.8446

Epoch 3/25
----------
train Loss: 0.6801 Acc: 0.7697
val Loss: 0.4092 Acc: 0.8566

Epoch 4/25
----------
train Loss: 0.5517 Acc: 0.8049
val Loss: 0.3655 Acc: 0.8765

Epoch 5/25
----------
train Loss: 0.4750 Acc: 0.8296
val Loss: 0.3202 Acc: 0.8845

Epoch 6/25
----------
train Loss: 0.4222 Acc: 0.8559
val Loss: 0.2876 Acc: 0.8924

Epoch 7/25
----------
train Loss: 0.3555 Acc: 0.8791
val Loss: 0.2477 Acc: 0.9084

Epoch 8/25
----------
train Loss: 0.3387 Acc: 0.8727
val Loss: 0.2970 Acc: 0.8964

Epoch 9/25
----------
train Loss: 0.3046 Acc: 0.8950
val Loss: 0.2688 Acc: 0.8964

Epoch 10/25
----------
train Loss: 0.2739 Acc: 0.9019
val Loss: 0.2389 Acc: 0.9124

Epoch 11/25
----------
train Loss: 0.2754 Acc: 0.9034
val Loss: 0.2442 Acc: 0.9163

Epoch 12/25
----------
train Loss: 0.2860 Acc: 0.9049
val Loss: 0.2559 Acc: 0.9163

E

In [78]:
save_model(densenet_photo, "densenet121_photo", densenet_photo_history)

del densenet_photo
del optimizer
del densenet_photo_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [79]:
#densenet mixed augmentation

densenet_mix = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT) #Load pretrained model
densenet_mix.classifier = nn.Linear(densenet_mix.classifier.in_features, num_classes)
densenet_mix = densenet_mix.to(device)

optimizer = optim.SGD(densenet_mix.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

densenet_mix, densenet_mix_history = train_model(
    model=densenet_mix,
    dataloaders=dataloaders_mix,
    dataset_size=dataset_size_mix,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.4855 Acc: 0.4344
val Loss: 0.8553 Acc: 0.7211

Epoch 2/25
----------
train Loss: 0.9583 Acc: 0.6548
val Loss: 0.5800 Acc: 0.8008

Epoch 3/25
----------
train Loss: 0.7976 Acc: 0.7093
val Loss: 0.4350 Acc: 0.8566

Epoch 4/25
----------
train Loss: 0.6934 Acc: 0.7583
val Loss: 0.4088 Acc: 0.8606

Epoch 5/25
----------
train Loss: 0.6245 Acc: 0.7727
val Loss: 0.3375 Acc: 0.8685

Epoch 6/25
----------
train Loss: 0.5660 Acc: 0.7989
val Loss: 0.2859 Acc: 0.9004

Epoch 7/25
----------
train Loss: 0.5268 Acc: 0.8123
val Loss: 0.2376 Acc: 0.9124

Epoch 8/25
----------
train Loss: 0.4901 Acc: 0.8271
val Loss: 0.2391 Acc: 0.9283

Epoch 9/25
----------
train Loss: 0.4409 Acc: 0.8484
val Loss: 0.2232 Acc: 0.9323

Epoch 10/25
----------
train Loss: 0.4018 Acc: 0.8534
val Loss: 0.2233 Acc: 0.9283

Epoch 11/25
----------
train Loss: 0.3673 Acc: 0.8687
val Loss: 0.2378 Acc: 0.9163

Epoch 12/25
----------
train Loss: 0.3691 Acc: 0.8687
val Loss: 0.2245 Acc: 0.9124

E

In [80]:
save_model(densenet_mix, "densenet121_mix", densenet_mix_history)

del densenet_mix
del optimizer
del densenet_mix_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [81]:
# ConvNeXt-Base
convnext_base = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)
in_features = convnext_base.classifier[2].in_features
convnext_base.classifier[2] = nn.Linear(in_features, num_classes)
convnext_base = convnext_base.to(device)

optimizer = optim.SGD(convnext_base.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

convnext_base, convnext_base_history = train_model(
    model=convnext_base,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.2680 Acc: 0.5493
val Loss: 0.7824 Acc: 0.7251

Epoch 2/25
----------
train Loss: 0.7864 Acc: 0.7325
val Loss: 0.5395 Acc: 0.8127

Epoch 3/25
----------
train Loss: 0.6060 Acc: 0.7935
val Loss: 0.4259 Acc: 0.8606

Epoch 4/25
----------
train Loss: 0.4970 Acc: 0.8311
val Loss: 0.3727 Acc: 0.8765

Epoch 5/25
----------
train Loss: 0.4716 Acc: 0.8474
val Loss: 0.3906 Acc: 0.8486

Epoch 6/25
----------
train Loss: 0.4270 Acc: 0.8524
val Loss: 0.2848 Acc: 0.9203

Epoch 7/25
----------
train Loss: 0.3859 Acc: 0.8628
val Loss: 0.2679 Acc: 0.9044

Epoch 8/25
----------
train Loss: 0.3420 Acc: 0.8811
val Loss: 0.2572 Acc: 0.9084

Epoch 9/25
----------
train Loss: 0.3477 Acc: 0.8811
val Loss: 0.2189 Acc: 0.9124

Epoch 10/25
----------
train Loss: 0.3526 Acc: 0.8836
val Loss: 0.3300 Acc: 0.8805

Epoch 11/25
----------
train Loss: 0.3612 Acc: 0.8787
val Loss: 0.2196 Acc: 0.9402

Epoch 12/25
----------
train Loss: 0.2684 Acc: 0.9059
val Loss: 0.2298 Acc: 0.9363

E

In [ ]:
save_model(convnext_base, "convnext_tiny_base", convnext_base_history)

del convnext_base
del optimizer
del convnext_base_history
torch.cuda.empty_cache

In [83]:
# ConvNeXt-geo
convnext_geo = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)
in_features = convnext_geo.classifier[2].in_features
convnext_geo.classifier[2] = nn.Linear(in_features, num_classes)
convnext_geo = convnext_geo.to(device)

optimizer = optim.SGD(convnext_geo.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

convnext_geo, convnext_geo_history = train_model(
    model=convnext_geo,
    dataloaders=dataloaders_geo,
    dataset_size=dataset_size_geo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.3553 Acc: 0.5002
val Loss: 0.8870 Acc: 0.6414

Epoch 2/25
----------
train Loss: 0.8332 Acc: 0.7107
val Loss: 0.6339 Acc: 0.7729

Epoch 3/25
----------
train Loss: 0.7275 Acc: 0.7578
val Loss: 0.4842 Acc: 0.8526

Epoch 4/25
----------
train Loss: 0.6291 Acc: 0.7855
val Loss: 0.3635 Acc: 0.8964

Epoch 5/25
----------
train Loss: 0.5547 Acc: 0.8034
val Loss: 0.2969 Acc: 0.9044

Epoch 6/25
----------
train Loss: 0.5547 Acc: 0.8133
val Loss: 0.2889 Acc: 0.9124

Epoch 7/25
----------
train Loss: 0.4979 Acc: 0.8266
val Loss: 0.3937 Acc: 0.8526

Epoch 8/25
----------
train Loss: 0.4716 Acc: 0.8316
val Loss: 0.5545 Acc: 0.7729

Epoch 9/25
----------
train Loss: 0.5079 Acc: 0.8192
val Loss: 0.2955 Acc: 0.8884

Epoch 10/25
----------
train Loss: 0.4231 Acc: 0.8509
val Loss: 0.3157 Acc: 0.8884

Epoch 11/25
----------
train Loss: 0.3602 Acc: 0.8787
val Loss: 0.2699 Acc: 0.8845

Epoch 12/25
----------
train Loss: 0.3344 Acc: 0.8831
val Loss: 0.2758 Acc: 0.9084

E

In [84]:
save_model(convnext_geo, "convnext_tiny_geo", convnext_geo_history)

del convnext_geo
del optimizer
del convnext_geo_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [85]:
# ConvNeXt-photo
convnext_photo = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)
in_features = convnext_photo.classifier[2].in_features
convnext_photo.classifier[2] = nn.Linear(in_features, num_classes)
convnext_photo = convnext_photo.to(device)

optimizer = optim.SGD(convnext_photo.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

convnext_photo, convnext_photo_history = train_model(
    model=convnext_photo,
    dataloaders=dataloaders_photo,
    dataset_size=dataset_size_photo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.2569 Acc: 0.5473
val Loss: 0.6731 Acc: 0.7530

Epoch 2/25
----------
train Loss: 0.8487 Acc: 0.7093
val Loss: 0.5112 Acc: 0.8327

Epoch 3/25
----------
train Loss: 0.6679 Acc: 0.7796
val Loss: 0.4344 Acc: 0.8446

Epoch 4/25
----------
train Loss: 0.6950 Acc: 0.7400
val Loss: 0.3642 Acc: 0.8805

Epoch 5/25
----------
train Loss: 0.5416 Acc: 0.8143
val Loss: 0.4416 Acc: 0.8367

Epoch 6/25
----------
train Loss: 0.5271 Acc: 0.8207
val Loss: 0.3122 Acc: 0.8964

Epoch 7/25
----------
train Loss: 0.4985 Acc: 0.8341
val Loss: 0.2927 Acc: 0.9044

Epoch 8/25
----------
train Loss: 0.4634 Acc: 0.8435
val Loss: 0.2606 Acc: 0.9163

Epoch 9/25
----------
train Loss: 0.4273 Acc: 0.8514
val Loss: 0.2834 Acc: 0.9124

Epoch 10/25
----------
train Loss: 0.4136 Acc: 0.8574
val Loss: 0.2617 Acc: 0.9084

Epoch 11/25
----------
train Loss: 0.3551 Acc: 0.8732
val Loss: 0.2327 Acc: 0.9243

Epoch 12/25
----------
train Loss: 0.4199 Acc: 0.8633
val Loss: 0.2537 Acc: 0.9203

E

In [86]:
save_model(convnext_photo, "convnext_tiny_photo", convnext_photo_history)

del convnext_photo
del optimizer
del convnext_photo_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>

In [87]:
# ConvNeXt-mixed
convnext_mixed = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)
in_features = convnext_mixed.classifier[2].in_features
convnext_mixed.classifier[2] = nn.Linear(in_features, num_classes)
convnext_mixed = convnext_mixed.to(device)

optimizer = optim.SGD(convnext_mixed.parameters(), lr=0.001, momentum =0.9, weight_decay=5e-4) #define optimizer

convnext_mixed, convnext_mixed_history = train_model(
    model=convnext_mixed,
    dataloaders=dataloaders_mix,
    dataset_size=dataset_size_mix,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25
)

Epoch 1/25
----------
train Loss: 1.4604 Acc: 0.4532
val Loss: 0.8250 Acc: 0.7689

Epoch 2/25
----------
train Loss: 0.9717 Acc: 0.6652
val Loss: 0.7228 Acc: 0.7570

Epoch 3/25
----------
train Loss: 0.8563 Acc: 0.6949
val Loss: 0.4497 Acc: 0.8765

Epoch 4/25
----------
train Loss: 0.7365 Acc: 0.7449
val Loss: 0.4202 Acc: 0.8645

Epoch 5/25
----------
train Loss: 0.6589 Acc: 0.7712
val Loss: 0.4256 Acc: 0.8685

Epoch 6/25
----------
train Loss: 0.6549 Acc: 0.7692
val Loss: 0.3482 Acc: 0.8725

Epoch 7/25
----------
train Loss: 0.6170 Acc: 0.7915
val Loss: 0.4201 Acc: 0.8606

Epoch 8/25
----------
train Loss: 0.5850 Acc: 0.7940
val Loss: 0.3405 Acc: 0.9004

Epoch 9/25
----------
train Loss: 0.5288 Acc: 0.8217
val Loss: 0.3015 Acc: 0.8964

Epoch 10/25
----------
train Loss: 0.4742 Acc: 0.8415
val Loss: 0.3491 Acc: 0.9044

Epoch 11/25
----------
train Loss: 0.4673 Acc: 0.8375
val Loss: 0.3289 Acc: 0.8805

Epoch 12/25
----------
train Loss: 0.4733 Acc: 0.8331
val Loss: 0.3056 Acc: 0.9203

E

In [88]:
save_model(convnext_mixed, "convnext_tiny_mixed", convnext_mixed_history)

del convnext_mixed
del optimizer
del convnext_mixed_history
torch.cuda.empty_cache

Model saved


<function torch.cuda.memory.empty_cache() -> None>